<a href="https://colab.research.google.com/github/CER-REC/esa-data-bank_banque-donnees-ees/blob/master/Codes/Section_05_VEC_Labeling_for_Tables/03_esa_table_vec_data_labeling_with_snorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labeling ESA tables with VEC labels

## Imports

In [10]:
!pip install snorkel --quiet
!pip install datasets --quiet
!pip install spacy --quiet
!pip install onnxruntime --quiet
!pip install better-profanity --quiet
!pip install pytorch-lightning==1.2.8 --quiet
!pip install transformers==4.5.1 --quiet
!pip install wandb --quiet

     |████████████████████████████████| 153kB 11.9MB/s 
     |████████████████████████████████| 1.8MB 29.7MB/s 
     |████████████████████████████████| 3.8MB 41.9MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 266kB 12.9MB/s 
     |████████████████████████████████| 245kB 45.4MB/s 
     |████████████████████████████████| 122kB 42.9MB/s 
     |████████████████████████████████| 4.5MB 15.7MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 849kB 13.2MB/s 
     |████████████████████████████████| 6.0MB 38.5MB/s 
     |████████████████████████████████| 276kB 37.3MB/s 
     |████████████████████████████████| 235kB 36.7MB/s 
     |████████████████████████████████| 829kB 29.4MB/s 
     |████████████████████████████████|

In [11]:
import pandas as pd
import numpy as np
import os
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.lf.nlp import nlp_labeling_function
from snorkel.preprocess import preprocessor
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.apply.dask import DaskLFApplier
from sklearn.model_selection import train_test_split
import spacy
from spacy.matcher import Matcher
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning, ModelPruning
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)

# weights and biases
import wandb

# lightning plus wandb
from pytorch_lightning.loggers import WandbLogger

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
Global seed set to 42


In [12]:
!nvidia-smi

Mon Jul 12 15:56:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Load CSVs

In [14]:
# esa_df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=False) # local
esa_df = pd.read_csv('/content/drive/MyDrive/data/esa_index_files/esa_index_with_table_text_no_labels.csv', index_col=False) # colab

In [15]:
esa_df.head()

,Unnamed: 0,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614

In [16]:
esa_df.drop(columns=['Unnamed: 0', 'Index'], inplace=True)

In [17]:
esa_df.head()

,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,L

In [10]:
esa_df['text'].head()

0    TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...
1    TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...
2    TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...
3    TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...
4    TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...
Name: text, dtype: object

In [12]:
snorkel_df = esa_df[['text', 'label']]

We will now go into Label Studio and label 500 examples. We will use these labels to create weak labels with Snorkel. Essentially, we won't need to label the entire dataset manually if we use Snorkel. However, we will need to look at the least confident examples and potentially relabel any mistakes.

In [ ]:
# labeled_df = pd.read_csv('./mydata/esa_index_labeled.csv') # local
labeled_df = pd.read_csv('./mydata/esa_index_labeled.csv') # colab

Here we create 4 dataset splits. What we call `train_ft_df` (Train Fine-Tune Set), `val_ft_df` (Validation Fine_tune Set), and `test_df` (Test Set) will contain labeled data. At the moment the examples are not labeled, but we will label them in Label Studio. The `train_df` is the unlabeled dataset we will be using Snorkel and a BERT model to label.

*   `fine_tune_df`: Labeled data we use for fine-tuning the bert model before putting model into a Snorkel Labeling Function.
*   `val_df`: This is our small dataset split we will use to measure the performance of the bert model trained with `fine_tune_df`.
*    `train_df`: As mentioned, the `train_df` is the unlabeled dataset we will be using Snorkel to label.
*    `test_df`: This is our dataset split for testing Snorkel's Label Model to get an estimate on how many labels were correctly labeled in our `train_df`.

In [ ]:
fine_tune_df, val_ft_df = train_test_split(labeled_df, 
                                           test_size=100, 
                                           random_state=42, 
                                           shuffle=True) # for fine-tuning BERT on subset of hand-labeled dataset
fine_tune_df, test_df = train_test_split(fine_tune_df, 
                                         test_size=500, 
                                         random_state=42, 
                                         shuffle=True) # for testing final model

## Snorkel Labels

In [13]:
# Encoding our labels for the labeling functions
ABSTAIN = -1
PHYSICAL_AND_METEOROLOGICAL_ENVIRONMENT = 0
SOIL_AND_SOIL_PRODUCTIVITY = 1
VEGETATION = 2
WATER_QUALITY_AND_QUANTITY = 3
FISH_AND_FISH_HABITAT = 4
WETLANDS = 5
WILDLIFE_AND_WILDLIFE_HABITAT = 6
SPECIES_AT_RISK = 7
GHG_EMISSIONS_AND_CLIMATE_CHANGE = 8
ASSESSMENT_OF_UPSTREAM_GHG_EMISSIONS = 9
AIR_EMISSIONS = 10
ACOUSTIC_ENVIRONMENT = 11
ELECTROMAGNETISM_AND_CORONA_DISCHARGE = 12
HUMAN_OCCUPANCY_AND_RESOURCE_USE = 13
HERITAGE_RESOURCES = 14
NAVIGATION_AND_NAVIGATION_SAFETY = 15
TRADITIONAL_LAND_AND_RESOURCE_USE = 16
SOCIAL_AND_CULTURAL_WELL_BEING = 17
HUMAN_HEALTH_AND_AESTHETICS = 18
INFRASTRUCTURE_AND_SERVICES = 19
EMPLOYMENT_AND_ECONOMY = 20
ENVIRONMENTAL_OBLIGATIONS = 21
RIGHTS_OF_INDIGENOUS_PEOPLES = 22

labels_list = [['Physical and meteorological environment',
                'Soil and soil productivity',
                'Vegetation',
                'Water quality and quantity',
                'Fish and fish habitat',
                'Wetlands',
                'Wildlife and wildlife habitat',
                'Species at Risk or Species of Special Status and related habitat',
                'Greenhouse gas (GHG) emissions and climate change',
                'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
                'Air emissions',
                'Acoustic environment',
                'Electromagnetism and Corona Discharge',
                'Human occupancy and resource use',
                'Heritage resources',
                'Navigation and navigation safety',
                'Traditional land and resource use',
                'Social and cultural well-being',
                'Human health and aesthetics',
                'Infrastructure and services',
                'Employment and economy',
                'Environmental Obligations',
                'Rights of Indigenous Peoples']]

In [14]:
labels_list

[['Physical and meteorological environment',
  'Soil and soil productivity',
  'Vegetation',
  'Water quality and quantity',
  'Fish and fish habitat',
  'Wetlands',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Greenhouse gas (GHG) emissions and climate change',
  'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
  'Air emissions',
  'Acoustic environment',
  'Electromagnetism and Corona Discharge',
  'Human occupancy and resource use',
  'Heritage resources',
  'Navigation and navigation safety',
  'Traditional land and resource use',
  'Social and cultural well-being',
  'Human health and aesthetics',
  'Infrastructure and services',
  'Employment and economy',
  'Environmental Obligations',
  'Rights of Indigenous Peoples']]

## Keywords

In [7]:
Physical_and_Meteorological_Environment = """Physical and Meteorological Environment
Precipitation
Snowfall
Wind
Mean temperatures
Slope
Geotechnical
Slumping
Subsidence
Weather
Erosion
Ice
Permafrost
Climate trend
ground
water erosion
wind erosion
acid-generating rock
temperature
physical
meteorological
landslides
mudflows
slumping
subsidence
seismicity
flooding
migrating watercourses
eroding banks
extreme weather events
peak flow regime
ice jams
acid rock
climate variability
ground conditions
thaw"""


Soil_and_Soil_Productivity = """Soil Productivity
Soil
Agriculture
Topsoil
Subsoil
Soil horizon
Drainage
Erosion
Contamination
CCME
Canadian Council of Ministers of the Environment
Soil compaction
Soil structure
Soil classification
Soil handling
Containment
reclamation
thickness of horizon
tilth
grubbing
soil quality
salinity
sediments
rocks
minerals
sand"""

Vegetation = """Vegetation
Plant
planting
Rare plant
Boreal
Grassland
Prairie
Forest
forested
Clearing
Diversity
Abundance
Ecological
Inventory
Community
communities
ecological community
plant community
Cover
Weeds
Invasive species
invasive plants
Seed mix
Herbicide
Tree
Growth
Old growth
Biodiversity
forestry
clubroot
wood
spruce
fir
birch
pine
aspen
tamarack
willow
beech
maple
black walnut
hickory
oak
redcedar
hemlock
Douglas-fir
genus
agricultural"""


Water_Quality_and_Quantity = """Water Quality and Quantity
Water
Surface water
Ground water
Runoff
Contamination
contaminant
Water use
Hydrologic
hydrological
Withdrawal
Discharge
Flow
Peak
Basin
Inter-basin transfer
Water Quality
Water quality testing
Water table
Containment
Sediment
sewer
waste
groundwater
groundwater-related
aquifers
streamflow
acquatics
river
hydrometric""" 

Fish_and_Fish_Habitat = """Fish and Fish Habitat
Fish
Fisheries and Oceans Canada
DFO
Fisheries and Oceans
Local fisheries
Trout
Authorization
Fisheries Act
Offsetting
Monitoring
Instream work
in-stream work
Timing window
Restricted activity period
Fish-bearing water body
Riparian
Aquatic
Aquatic invasive species
Spawning
Migrating
Eggs
Fry
Sport fishery
Spawning deterrent
Stream
River
In-stream
instream
Wetted width
substrate
whale
marine mammal
salmon
oncorhynchus
walleye
pike
crappie
redhorse
pumpkinseed fish
bowfin
bass
catfish
sunfish
bluegill
spotted gar
muskellunge
ruffe
yellow perch
shiner
sucker
whitefish
cisco"""

Wetlands = """Wetlands
Class (wetland class)
wetland
Bog
Fen
Marsh
Swamp
Shallow water
Intermittent
Permanent
Non-permanent
Wetland function
Offsetting
Hydrological function
Drainage area
Canadian wetland classification system
Federal policy on wetland conservation
Conservation
Wetland monitoring
Reclamation
Compensation
water recharge
potholes
ponds
peatbogs
mires""" 

Wildlife_and_Wildlife_Habitat = """Wildlife and Wildlife Habitat
nocturnal
bat trees
hibernation
migratory birds
MBCA
migratory birds convention act
Nesting
Restricted activity period
Timing window
Mammal
Ungulate
Amphibian
Reptile
Breeding
Den
denning
Wintering
overwintering
Migration
Hibernaculum
hibernacula
Riparian habitat
Old growth habitat
Sensitive period
Sensory disturbance
Mortality
Mortality risk
Habitat alteration
Habitat loss
Habitat destruction
Range
Population
Distribution
Sanctuary
sanctuaries
Important Bird Area
Bat
caribou
bird
goose
amphibian
swans
frogs
toads
salamanders
newts
caecilians
furbearing
reptile
invertebrate
rattlesnake
snake
waterbird
bear
moose
bat
owl"""

Species_at_Risk = """Species at Risk
Species of Special Status
Rare
Sensitive
SARA
Species At Risk Act
COSEWIC
Committee on the Status of Endangered Wildlife in Canada
ECCC
Environment and Climate Change Canada
Critical habitat
Designation
Threshold
Schedule 1
At Risk
Endangered
Critical timing window
Restricted activity period
DFO
Fisheries and Oceans Canada
Canadian Wildlife Service
CWS
Recovery Strategy
Action Plan
Status
Listed
Permit
Offset
offsetting
Compensation
Caribou
Bat
special conservation status"""

Air_Emissions = """Air Emissions
CAC
criteria air contaminant
Emissions
Ground-level
Receptor
Model
modelled
modelling
Construction equipment
Vehicle
vehicular emissions
CCME
Volatile organic compounds
Combustion
Leak
Fugitive emissions
Detection
Flaring
Incinerating
incineration
Smoke
Venting
Pollute
pollutant
National Pollutant Release Inventory
Exceedance
Permit
Release
Ambient
Air quality
Hydrogen sulphide
H2S
particulate
so2"""

GHG_Emissions_and_Climate_Change = """GHG Emissions and Climate Change
greenhouse
greenhouse gas
green house gas
climate change
point source
area source
venting
fugitive
release
leak
burning
assumption
offset
carbon dioxide
CO2
CO2 equivalent
Target
Reduction
Percentage
Hinders
Net zero
net-zero
Reduce
Combustion emissions
International Standards Organization
ISO
methane
ghg
ozone
global warming"""

Assessment_of_Upstream_GHG_Emissions = """Assessment of Upstream GHG Emissions
upstream
quantitative
throughput
net zero
net-zero
Environment and Climate Change Canada (ECCC)
Venting
Threshold
CO2
Carbon dioxide
CO2 equivalent
Methane
emissions
steam
hydrogen
combustion
fugitive
venting
flaring"""

Acoustic_Environment = """Acoustic Environment
Sound
Noise
Receptor
Equipment
Frequency
Inaudible
Audible
Decibel
Notification
Noise control
Noise management
Model
modelling
Monitoring
monitor
db
acoustic"""


Environmental_Obligations = """Environmental Obligations
MBCA
migratory birds convention act
SARA
Species at risk act
DFO
Fisheries and Oceans Canada
Federal Wetland Policy
Hinder
Federal
Provincial
Territorial
International
Policy
Plan
Framework
Wildlife
Vegetation
Fish
Species at risk
Plant
law
legislation
regulations"""

Traditional_Land_and_Resource_Use = """Traditional Land and Resource Use
TLRU
traditional
Traditional ecological knowledge
Traditional Knowledge
Indigenous Knowledge
Aboriginal Knowledge
Aboriginal
Hunt
hunting
fishing
Harvest
harvesting
Culturally significant
Culturally modified tree
Gather
Berries
Medicine
Berry picking
Indigenous
Elder
Knowledge Keeper
Trapping
trap
Ceremony
ceremonies
Medicinal
Cultural
Old growth
Spirit Bear
Spirit animal
spiritual
sacred area
sacred sites
metis
Métis
first nations
shxw’ōwhámel
lheidlit’enneh
whispering pines first nation
inuit
elders
kumik elder lodge
tribal
Abenaki
Innu
Montagnais-Naskapi
Oneida
Ahousaht
Interior Salish
Onondaga
Algonquin
Inuinnait
Copper Inuit
Pacheenaht
Assiniboine
Inuvialuit
Mackenzie Inuit
Petun
Atikamekw
Kainai
Piikani
Peigan
Baffin Island Inuit
K'asho Got'ine
Saldermiut Inuit
Beothuk
Kaska Dena
Sahtu Got'ine
Bearlake
Blackfoot Confederacy
Blackfoot
Kivallirmiut
Caribou Inuit
Secwepemc
Shuswap
Cayuga
Ktunaxa
Kootenay
Sekani
Central Coast Salish
Kwakwaka'wakw
Kwakiutl
Seneca
Coast Salish
Kyuquot and Checleseht
Shuta Got'ine
Cree
Labradormiut
Labrador Inuit
Siksika
Dakota
Lilwat
Lillooet
Slavey
Dakelh
Lingit
Tlingit
Stoney-Nakoda
Dane-zaa
Beaver
Syilx
Okanagan
Dene
Mi'kmaq
Tagish
Denesuline
Chipewyan
Mohawk
Tahltan
Ditidaht
Mowachaht-Muchalaht
Tla-o-qui-aht
Clayoquot
Ehattesaht
Nahani
Tlicho
Dogrib
Gitxsan
Gitksan
Netsilingmiut
Netsilik Inuit
Toquaht
Gwich'in
Neutral Confederacy
Tr'ondëk Hwëch'in (Han)
Haida
Nicola-Similkameen
Tseshaht
Sheshaht
Haisla
Kitamaat
Nisga'a
Tsilhqot'in
Chilcotin
Haudenosaunee
Six Nations
Iroquois
Nlaka'pamux
Thompson
Tsimshian
Heiltsuk
Northern Georgia Strait Coast Salish
Tsuut'ina
Sarcee
Hesquiaht
Nuchatlaht
Tutchone
Hupacasath
Opetchesaht
Nunavimmiut
Ungava Inuit
Uchucklesaht
Huu-ay-aht
Nuu-chah-nulth
Ucluelet
Huron-Wendat
Nuxalk
Bella Coola
Wolastoqiyik
Maliseet
Iglulingmuit
Iglulik Inuit
Odawa
Wetal
Tsetsaut
Inuit
Ojibwa
Yellowknives
popkum first nation
leq’á:mel first nation
alexander first nation
samson cree first nation
o’chiese first nation
ermineskin cree nation
enoch cree nation
indian
eskimo"""

Electromagnetism_and_Corona_Discharge = """Electromagnetism and Corona Discharge
voltage
ozone concentration
eletric
corona
discharge
magnetic
power line
electromagnetic
signals
maximum load
induction
frequency
inteference
radio
television
foul weather
ambient conditions
240 kV"""

Human_Occupancy_and_Resource_Use = """Human Occupancy and Resource Use
Human Occupancy and Resource Use
residents
human Occupancy
resource use
consultation
livestock
human
rural
urban
residential
reserve
crops
orchards
vineyards
agriculture
recreation
park
scenic
parks canada
conservation area
international biological program
ecological reserves
preserves
industrial
commercial
agreement forests
timber sales area
controlled forest
managed forest
registered hunting
recognized hunting
trapping
guiding areas
commercial fishing
sport finishing
water reserves
water licenses
water supply
municipal
infrastructure
rail
navigable waterways
local"""

Heritage_Resources = """Heritage Resources
Heritage Resources
Heritage
Archaeology
Archeology
Archaeological
Archeological
Paleontology
Paleontological
Historic
Historic resource
Historic site
Hunting camp
Trail
Culturally significant
Dig site
Archaeologist
Archeologist
undiscovered
architectural
grave site
burial site
medicine wheel
culturally modified tree
CMT
archaeological report
archeological report
archaeological assessment
archeological assessment
pre-contact
post-contact
human remains
Permit
license
licence
clearance
Heritage Conservation Branch
Ontario ministry of tourism culture and sport
Heritage resources act
Alberta Ministry of Culture Multiculturalism and Status of Women
British Columbia Archeology Branch
Heritage conservation act"""

Navigation_and_Navigation_Safety = """Navigation and Navigation Safety
Navigation
Navigation protection activities
Guide lines
Guide wires
Signage
Waterway
Crossing
crossing plan
navigation
impacting Navigation
dewatering of navigable waters
navigable waters
navigable waterway
navigable
navigate
waterway
watercourse
watercourse crossing
water crossing
crossing
crossing methodology
horizontal directional drilling
HDD
bridge
temporary
permanent
marine
marine terminal
waterway user
recreational waterway user
navigational use
river
creek
tributary
tourism
tourist
guide outfitter
outfitter
angler
canoe
kayak
boat
sailing
sail
Fisheries and Oceans Canada
Navigation Protection Act
navigable watercourse
navigation Safety
scheduled waters
non-scheduled waters
recreation-related navigation
commercial-related navigation
watercourse users
waterway users
watercourse Crossing
crossing method
trenchless crossing
trenched crossing
navigation hazard
exposed instream
buoyancy issues
upstream
downstream
warning signs
warning Signage
instream
temporary vehicle crossing
bed
banks
preconstruction contours
hydraulic characteristics
timing windows
erosion and sediment control
runoff
temporary crossing structure
fording
streambank
streambed
side containment"""

Social_and_Cultural_Well_Being = """Social and Cultural Well-Being
Social
Routing
socio-cultural
cultural
well-being
well being
families
workers
residents
community
traditions
alcohol
substance abuse
stresses
household cohesion
illegal
disruptive activities
privacy
inhabited
population
human behaviour
human behavior
workforce
peak workforce
mobile workforce
discipline measures
traffic control management
project schedule
Code of conduct policy
alcohol and drug policy"""

Human_Health_and_Aesthetics = """Human Health and Aesthetics
toxic
nuisances
health
Aesthetics
human receptors
CCME Guidelines
AER Directive 038
AUC Rule 012
release assessment
exposure assessment
dose-response
risk characterization
mental
Social
well-being
well being
stressors
emotional
public Safety
accidents
visual
obstruction of view
view points
angle of vision
quality of life"""

Infrastructure_and_Services = """Infrastructure and Services
Hospital
Urgent Care
ambulance
Fire services
Fire response
protective services
police services
Emergency response time
Emergency response
Hotel
Motel
RCMP
Royal Canadian Mounted police
medical response personnel
healthcare
social services
Local commercial accommodation
local accommodation
existing accommodation
worker accommodation
campground
Recreational Camp sites
recreational Resources
camp sites
Municipal waste
Municipal wateruse
Municipal water use
waste
contingency plan
traffic control
multi-passenger vehicles
restrict access
service providers
chemical waste
solid waste
liquid waste
landfills
industrial waste
non-hazardous waste
transfer stations
hazardous waste facilities
wastewater treatment facilities
recycling facilities
highways
roads
airports
911 dispatch services
Local commercial accommodation
Camp sites
Recreational Camp sites
railway
rail
roadway
road
highway
traffic
traffic flow
traffic usage levels
traffic patterns
pipeline
water main
water supply
sewage line
waste water
waste disposal
navigable waterway
powerline
power line
existing
pre-existing
preexisting
local services
regional services
services
accommodation
camping
facilities
recreation
recreational
ammenities
community services
essential services
emergency services
health care services
social services
police
fire
fire fighting
fire-fighting
firefighting
EMT
response time
healthcare
health care
hospital
housing
educational facilities
school
university
college
transportation
access
construction access
land access
right of way
right-of-way
ROW
temporary workspace
temporary work space
TWS"""

Employment_and_Economy = """Employment and Economy
Employment opportunities
Business opportunities
contracting opportunities
Project contracting
local contracting
subcontracting
Indigenous employment
Aboriginal employment
Aboriginal participation plan
Indigenous participation plan
Aboriginal businesses
Indigenous businesses
direct employment
prime contractor
local Business
local Economy
local economies
unemployment rate
employment rate
educational level
post-secondary
high school
certificate
diploma
degree
Economy
jobs
personnel
cotractors
workers
workforce
staff
labour force
labor force
economic well-being
procurement
tax revenue
taxes
revenue
Major industries
Primary industries
Key industrial sectors
tourism
mining
quarrying
oil and Gas
gas extraction
agriculture
forestry
fishing
hunting
construction
public administration
retail trade
temporary workforce
permanent workforce
permanent part-time Employment
permanent full-time Employment
temporary part-time Employment
temporary full-time Employment
employment
contracting
contract
procurement
ordering
training
training programs
education
opportunity
capacity
labor
labour
development plan
labour services
economic participation
project requirements
dollar value
contract value
worker
workforce
work force
revenue
tax levee
employment level
unemployment level
unemployment
education level
skill level
economic condition
direct revenue
indirect revenue
hardship
displacement
economic benefits plan
cooperation agreement
bid
qualification
partnership
collaboration
distribution
agreement
outreach
commitment
financial
community investment
investment
development
benefits
monitor
Indigenous monitoring"""

Rights_of_Indigenous_Peoples = """Rights of Indigenous Peoples
Indigenous and Treaty Rights
Aboriginal and Treaty Rights
Treaty Rights
Indigenous Rights
Aboriginal Rights
potential rights
established rights
asserted rights
protected rights
section 35 rights
Indian Act
Constitution Act, 1982
Constitution Act
section 35
s. 35
Constitution
constitutionally
Indigenous
Aboriginal
Native
Indian
Métis
Metis
Inuit
Inuk
Peoples
Communities
Nation
Band
Tribe
Settlement
Treaty
Treaty Lands
Crown Land
Indigenous Land
Traditional Land
territory
Traditional Territory
Indigenous Knowledge
Oral Indigenous Knowledge
Traditional Knowledge
IK
OIK
TK
Elder
knowledge keeper
knowledge holder
rights-bearing
engagement
Indigenous engagement
Aboriginal engagement
Crown
Crown Consultation
duty to consult
agent of the Crown
early engagement
CER Early Engagement Guide
Indigenous and Northern Affairs Canada
INAC
Crown-Indigenous Relations and Northern Affairs Canada
CIRNAC
Indigenous Services Canada
ISC
confidential
confidentiality
infringe
infringement
exercise rights
practice rights
values
customs
traditions
practices
access
access to lands
access to resources
travel ways
land availability
resource availability
Indigenous protcols
Indigenous laws
governancy system
Indigenous participation
Indigenous monitoring
Reconciliation
Truth and Reconciliation
TRC
Calls to Action
missing and murdered
MMIW
MMIWG
residential school
United Nations Declaration on the Rights of Indigenous Peoples
UNDRIP
Traditional Knowledge
Indigenous Knowledge
Aboriginal Knowledge
Aboriginal
Hunt
hunting
fishing
Harvest
harvesting
Culturally significant
Culturally modified tree
Gather
Berries
Medicine
Berry picking
Indigenous
Elder
Trapping
engagement
trap
Ceremony
ceremonies
Medicinal
Cultural
First Peoples
indian act
treaty
rights-bearing
indigenous rights
reserves
aboriginal
indigenous
metis
Métis
first nations
shxw’ōwhámel
lheidlit’enneh
whispering pines first nation
inuit
elders
kumik elder lodge
tribal
Abenaki
Innu
Montagnais-Naskapi
Oneida
Ahousaht
Interior Salish
Onondaga
Algonquin
Inuinnait
Copper Inuit
Pacheenaht
Assiniboine
Inuvialuit
Mackenzie Inuit
Petun
Atikamekw
Kainai
Piikani
Peigan
Baffin Island Inuit
K'asho Got'ine
Saldermiut Inuit
Beothuk
Kaska Dena
Sahtu Got'ine
Bearlake
Blackfoot Confederacy
Blackfoot
Kivallirmiut
Caribou Inuit
Secwepemc
Shuswap
Cayuga
Ktunaxa
Kootenay
Sekani
Central Coast Salish
Kwakwaka'wakw
Kwakiutl
Seneca
Coast Salish
Kyuquot and Checleseht
Shuta Got'ine
Cree
Labradormiut
Labrador Inuit
Siksika
Dakota
Lilwat
Lillooet
Slavey
Dakelh
Lingit
Tlingit
Stoney-Nakoda
Dane-zaa
Beaver
Syilx
Okanagan
Dene
Mi'kmaq
Tagish
Denesuline
Chipewyan
Mohawk
Tahltan
Ditidaht
Mowachaht-Muchalaht
Tla-o-qui-aht
Clayoquot
Ehattesaht
Nahani
Tlicho
Dogrib
Gitxsan
Gitksan
Netsilingmiut
Netsilik Inuit
Toquaht
Gwich'in
Neutral Confederacy
Tr'ondëk Hwëch'in (Han)
Haida
Nicola-Similkameen
Tseshaht
Sheshaht
Haisla
Kitamaat
Nisga'a
Tsilhqot'in
Chilcotin
Haudenosaunee
Six Nations
Iroquois
Nlaka'pamux
Thompson
Tsimshian
Heiltsuk
Northern Georgia Strait Coast Salish
Tsuut'ina
Sarcee
Hesquiaht
Nuchatlaht
Tutchone
Hupacasath
Opetchesaht
Nunavimmiut
Ungava Inuit
Uchucklesaht
Huu-ay-aht
Nuu-chah-nulth
Ucluelet
Huron-Wendat
Nuxalk
Bella Coola
Wolastoqiyik
Maliseet
Iglulingmuit
Iglulik Inuit
Odawa
Wetal
Tsetsaut
Inuit
Ojibwa
Yellowknives
popkum first nation
leq’á:mel first nation
alexander first nation
samson cree first nation
o’chiese first nation
ermineskin cree nation
enoch cree nation
indian
eskimo"""

In [8]:
keywords = [Physical_and_Meteorological_Environment,
            Soil_and_Soil_Productivity,
            Vegetation,
            Water_Quality_and_Quantity,
            Fish_and_Fish_Habitat,
            Wetlands,
            Wildlife_and_Wildlife_Habitat,
            Species_at_Risk,
            GHG_Emissions_and_Climate_Change,
            Assessment_of_Upstream_GHG_Emissions,
            Air_Emissions,
            Acoustic_Environment,
            Electromagnetism_and_Corona_Discharge,
            Human_Occupancy_and_Resource_Use,
            Heritage_Resources,
            Navigation_and_Navigation_Safety,
            Traditional_Land_and_Resource_Use,
            Social_and_Cultural_Well_Being,
            Human_Health_and_Aesthetics,
            Infrastructure_and_Services,
            Employment_and_Economy,
            Environmental_Obligations,
            Rights_of_Indigenous_Peoples]

keywords = [x.lower().split('\n') for x in keywords]

In [9]:
keywords[0]

['physical and meteorological environment',
 'precipitation',
 'snowfall',
 'wind',
 'mean temperatures',
 'slope',
 'geotechnical',
 'slumping',
 'subsidence',
 'weather',
 'erosion',
 'ice',
 'permafrost',
 'climate trend',
 'ground',
 'water erosion',
 'wind erosion',
 'acid-generating rock',
 'temperature',
 'physical',
 'meteorological',
 'landslides',
 'mudflows',
 'slumping',
 'subsidence',
 'seismicity',
 'flooding',
 'migrating watercourses',
 'eroding banks',
 'extreme weather events',
 'peak flow regime',
 'ice jams',
 'acid rock',
 'climate variability',
 'ground conditions',
 'thaw']

## Stemming and Removing Stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

stemmer = PorterStemmer()

for i, label_keywords in enumerate(keywords):
    keywords[i] = [w for w in label_keywords if w not in stopwords.words("english")]
    keywords[i] = [stemmer.stem(w) for w in keywords[i]]

keywords

[['physical and meteorological environ',
  'precipit',
  'snowfal',
  'wind',
  'mean temperatur',
  'slope',
  'geotechn',
  'slump',
  'subsid',
  'weather',
  'eros',
  'ice',
  'permafrost',
  'climate trend'],
 ['soil',
  'agricultur',
  'topsoil',
  'subsoil',
  'soil horizon',
  'drainag',
  'eros',
  'contamin',
  'ccme ',
  'canadian council of ministers of the environ',
  'soil compact',
  'soil structur',
  'soil classification ',
  'soil handl',
  'contain',
  'reclam'],
 ['veget',
  'plant',
  'plant',
  'rare pl',
  'boreal',
  'grassland',
  'prairie ',
  'forest',
  'forest',
  'clear',
  'divers',
  'abund',
  'ecological ',
  'inventory ',
  'classif',
  'commun',
  'commun',
  'ecological commun',
  'plant commun',
  'cover',
  'weed',
  'invasive speci',
  'invasive plants ',
  'seed mix',
  'herbicid',
  'tree',
  'growth ',
  'old growth',
  'biodivers'],
 ['water',
  'surface wat',
  'ground wat',
  'runoff',
  'contamin',
  'contamin',
  'water us',
  'hydrolog'

## Loading the Labeled Data from the Zero-Shot Classification

In [6]:
import pickle
pickled_dataset = 'zero_shot_vcs_1k_for_labeling'
# infile = open(f'./data/{pickled_dataset}.pkl', 'rb') # local
infile = open(f'/content/drive/MyDrive/data/esa_index_files/zero_shot_vcs_1k_for_labeling.pkl', 'rb') # colab
zs_vcs_dict_list = pickle.load(infile)
infile.close()

In [7]:
zs_vcs_dict_list[0:2]

[{'labels': ['Wildlife and wildlife habitat',
   'Water quality and quantity',
   'Fish and fish habitat',
   'Acoustic environment',
   'Physical and meteorological environment',
   'Vegetation',
   'Soil and soil productivity',
   'Wetlands',
   'Heritage resources',
   'Environmental Obligations',
   'Social and cultural well-being',
   'Air emissions',
   'Infrastructure and services',
   'Traditional land and resource use',
   'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
   'Species at Risk or Species of Special Status and related habitat',
   'Rights of Indigenous Peoples',
   'Greenhouse gas (GHG) emissions and climate change',
   'Employment and economy',
   'Human occupancy and resource use',
   'Navigation and navigation safety',
   'Electromagnetism and Corona Discharge',
   'Human health and aesthetics'],
  'scores': [0.9908032417297363,
   0.9892772436141968,
   0.9859840869903564,
   0.9473797082901001,
   0.9381691217422485,
   0.88502639532

In [22]:
filtered_zs_vcs = []

for zs_vcs_i in zs_vcs_dict_list[:10]:
    num_of_labels = len([filtered_score for filtered_score in zs_vcs_i['scores'] if filtered_score >= 0.8])
    filtered_zs_vcs.append(zs_vcs_i['sequence'], 
                           zs_vcs_i['labels'][:num_of_labels], 
                           zs_vcs_i['scores'][:num_of_labels])

filtered_zs_vcs

TypeError: ignored

In [23]:
[filtered_score for filtered_score in zs_vcs_dict_list[0]['scores'] if filtered_score >= 0.8]

[0.9908032417297363,
 0.9892772436141968,
 0.9859840869903564,
 0.9473797082901001,
 0.9381691217422485,
 0.8850263953208923,
 0.8406775593757629]

In [24]:
zs_vcs_dict_list[0]['scores'][:7]

[0.9908032417297363,
 0.9892772436141968,
 0.9859840869903564,
 0.9473797082901001,
 0.9381691217422485,
 0.8850263953208923,
 0.8406775593757629]

## Creating the Snorkel Functions

In [ ]:
from snorkel.labeling import labeling_function

keywords = [Physical_and_Meteorological_Environment,
            Soil_and_Soil_Productivity,
            Vegetation,
            Water_Quality_and_Quantity,
            Fish_and_Fish_Habitat,
            Wetlands,
            Wildlife_and_Wildlife_Habitat,
            Species_at_Risk,
            GHG_Emissions_and_Climate_Change,
            Assessment_of_Upstream_GHG_Emissions,
            Air_Emissions,
            Acoustic_Environment,
            Electromagnetism_and_Corona_Discharge,
            Human_Occupancy_and_Resource_Use,
            Heritage_Resources,
            Navigation_and_Navigation_Safety,
            Traditional_Land_and_Resource_Use,
            Social_and_Cultural_Well_Being,
            Human_Health_and_Aesthetics,
            Infrastructure_and_Services,
            Employment_and_Economy,
            Environmental_Obligations,
            Rights_of_Indigenous_Peoples]


# TODO: Create a labeling for each label
# What does L_train look like? It adds up all of the counts together and then does probabilities?
# Then that means that means that the total sum of all labels will be 1.
# Which means you need to make the cut off quite low, like 0.2.


@labeling_function
def lf_keyword_physical(x, phys, label):
    if any(word in x.text for word in keywords):
        return label
    return ABSTAIN

def make_keyword_lf(keywords, label=TOXIC):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )

keyword_physical = make_keyword_lf(keywords=["please", "plz", "pls", "pl", "Please read this", "Please read the", "Please read"], label=NONTOXIC)
keyword_soil = make_keyword_lf(keywords=["thanks", "thank you", "thx", "tx"], label=NONTOXIC)
keyword_vegetation = make_keyword_lf(keywords=keyword_bad, label=TOXIC)
keyword_Water
keyword_Fish_and_Fish_Habitat,
keyword_Wetlands,
keyword_Wildlife_and_Wildlife_Habitat,
keyword_Species_at_Risk,
keyword_GHG_Emissions_and_Climate_Change,
keyword_Assessment_of_Upstream_GHG_Emissions,
keyword_Air_Emissions,
keyword_Acoustic_Environment,
keyword_Electromagnetism_and_Corona_Discharge,
keyword_Human_Occupancy_and_Resource_Use,
keyword_Heritage_Resources,
keyword_Navigation_and_Navigation_Safety,
keyword_Traditional_Land_and_Resource_Use,
keyword_Social_and_Cultural_Well_Being,
keyword_Human_Health_and_Aesthetics,
keyword_Infrastructure_and_Services,
keyword_Employment_and_Economy,
keyword_Environmental_Obligations,
Rights_of_Indigenous_Peoples

In [ ]:
applier = PandasLFApplier(lfs)
L_train = applier.(df=train_df)



In [ ]:
L_test = applier.apply(df=test_df)

## Saving `L_train` and `L_test`

In [ ]:
import pickle

L_train_filename = 'L_train_toxic_finetuned_bert_1000ex_256tokens_8bs.pkl'

with open(L_train_filename,'wb') as f:
  pickle.dump(L_train, f)

In [ ]:
import pickle
L_train_filename = 'L_train_toxic_finetuned_bert_1000ex_256tokens_8bs.pkl'
with open(L_train_filename,'rb') as f:
  L_train = pickle.load(f)
  print(L_train.shape)

(15200, 8)


In [ ]:
L_train

array([[-1, -1, -1, ...,  0, -1,  1],
       [-1, -1, -1, ..., -1,  1,  0],
       [-1,  0, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1,  1,  1],
       [-1,  0, -1, ..., -1,  1,  1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [ ]:
import pickle

L_test_filename = 'L_test_toxic_finetuned_bert_1000ex_256tokens_8bs.pkl'

with open(L_test_filename,'wb') as f:
  pickle.dump(L_test, f)

In [ ]:
import pickle
L_test_filename = 'L_test_toxic_finetuned_bert_1000ex_256tokens_8bs.pkl'
with open(L_test_filename,'rb') as f:
  L_test = pickle.load(f)
  print(L_test.shape)

(3420, 8)


## Testing our Label Model

In [ ]:
from snorkel.labeling import LFAnalysis
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
contains_work_of_art,0,[0],0.061842,0.057829,0.013618
contains_entity,1,[0],0.304342,0.246382,0.084474
contains_email,2,[0],0.001645,0.001513,0.000197
contains_url,3,[0],0.031184,0.025132,0.006184
keyword_please,4,[0],0.321053,0.252434,0.087434
keyword_thanks,5,[0],0.084539,0.065132,0.010263
keyword_DIE,6,[1],0.199276,0.195855,0.091316
lf_bert_sentiment,7,"[0, 1]",0.512763,0.295132,0.137237


In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=23, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [ ]:
Y_test = test_df.label.values
LFAnalysis(L_test, lfs).lf_summary(Y_test)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
contains_work_of_art,0,[0],0.054678,0.050877,0.011988,127,60,0.679144
contains_entity,1,[0],0.295029,0.234211,0.083333,635,374,0.629336
contains_email,2,[0],0.002632,0.001754,0.000585,6,3,0.666667
contains_url,3,[0],0.031287,0.025439,0.005556,76,31,0.710280
keyword_please,4,[0],0.309064,0.240936,0.086550,649,408,0.614002
keyword_thanks,5,[0],0.073392,0.053216,0.007602,212,39,0.844622
keyword_DIE,6,[1],0.200292,0.196491,0.085088,679,6,0.991241
lf_bert_sentiment,7,"[0, 1]",0.519591,0.301754,0.134503,1362,415,0.766460


## Saving Label Model

In [ ]:
files.download(L_train_filename) 
files.download(L_test_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from snorkel.labeling.model import MajorityLabelVoter
from sklearn.preprocessing import MultiLabelBinarizer

Y = labels_list

# fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform(Y)

# create a majority vote model and predict
majority_model = MajorityLabelVoter(cardinality=23)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1]])

In [ ]:
from snorkel.labeling import MajorityLabelVoter
from sklearn.preprocessing import MultiLabelBinarizer

Y = labels_list

# fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform(Y)

# create a majority vote model and predict
majority_model = MajorityLabelVoter(cardinality=23)
predictions = majority_model.predict_proba(L=L_test)
df_multilabel = pd.DataFrame()
df_multilabel['predict_proba'] = predictions.tolist()

# get all the non zero indices which are the multi labels
df_multilabel['multi_labels'] = df_multilabel['predict_proba'].apply(lambda x: np.nonzero(x)[0])
    
#transform to mlb for classification report
df_multilabel['mlb_pred'] = df_multilabel['multi_labels'].apply(lambda x: mlb.transform([x])[0])
print(df_multilabel.head())

#convert to str in order to see how many multi labels did we gain
multi_label_string = df_multilabel.multi_labels.apply(lambda x: ", ".join(le.inverse_transform(x)))
print(multi_label_string.value_counts()[:50])

# print some metrics using classification report 
y_pred = df_multilabel.mlb_pred.apply(lambda x: list(x)).to_numpy().tolist()
y_true = mlb.transform(Y.values).tolist()
print(classification_report(y_true, y_pred, target_names = mlb.classes_))